In [105]:
import pandas as pd
import subprocess
import os
from ucsc_genomes_downloader import download_genome

In [125]:
from math import floor, ceil
def value_to_bed(base_pairs:int):
    def _value_to_bed(value:str)->str:
        chromosome, regions  = value.split("Mod_")[1].split("_")[0].split(":")
        start, end = regions.split("-")
        start = int(start)
        end = int(end)
        delta = base_pairs - (end-start)
        if delta < 0:
            raise ValueError("Base pairs window is smaller than given file window.")
        start = start-int(floor(delta/2))
        end = end+int(ceil(delta/2))
        return f"{chromosome},{start},{end},{chromosome}:{start}-{end}"
    return _value_to_bed

In [126]:
def to_bed(df:pd.DataFrame, region:str, base_pairs:int)->pd.DataFrame:
    df[region] = df[region].apply(value_to_bed(base_pairs))
    df = pd.concat([
        df[region].str.split(',', expand=True),
        df.drop(columns=region)
    ], axis=1)
    df.columns = ["chr", "start", "end", "id", "activity_ratio"]
    return df

In [127]:
file_name = "liver_enhancers"
genome = "hg19"
base_pairs = 200
region = f"{file_name}.bed"
df = pd.read_csv(f"{file_name}.tsv.gz", sep="\t")
download_genome(genome)
to_bed(df, "Region", base_pairs).to_csv(region, index=False, sep="\t", header=False)
goal = f"{file_name}.fa"
subprocess.run([
    "fastaFromBed",
    "-fi",
    f"{genome}.fa",
    "-bed",
    region,
    "-fo",
    goal
])

CompletedProcess(args=['fastaFromBed', '-fi', 'hg19.fa', '-bed', 'liver_enhancers.bed', '-fo', 'liver_enhancers.fa'], returncode=0)